Define Utility Functions


In [6]:
def print_matrix(mat, row_names=None, col_names=None):
    """
    Prints a matrix with optional row and column names.

    :param mat: The matrix to be printed.
    :param row_names: List of row names (labels). If None, no row names are printed.
    :param col_names: List of column names (labels). If None, no column names are printed.
    """
    if col_names is None:
        col_names = []
    if row_names is None:
        row_names = []

    # Print column names
    if col_names:
        print(" " * (max(len(name) for name in col_names) + 1) + " ".join(f"{name:>5}" for name in col_names))

    # Print matrix rows with row names
    for i, row in enumerate(mat):
        if row_names:
            print(f"{row_names[i]:>5}", end=" ")
        print(" ".join(f"{val:>5}" for val in row))

def zeros(rows, cols):
    """Creates a matrix of zeroes with specified rows and columns."""
    return [[0] * cols for _ in range(rows)]

Define Constants and Scoring Functions

In [7]:
# Constants for scoring
GAP_PENALTY = -2
MATCH_AWARD = 10
MISMATCH_PENALTY = -9

def match_score(alpha, beta):
    """Returns the score for aligning two bases."""
    if alpha == beta:
        return MATCH_AWARD
    elif alpha == '-' or beta == '-':
        return GAP_PENALTY
    else:
        return MISMATCH_PENALTY


Define Needleman-Wunsch Function

In [8]:
def needleman_wunsch(seq1, seq2):
    """Performs Needleman-Wunsch sequence alignment."""
    n = len(seq1)
    m = len(seq2)

    # Initialize score matrix
    score = zeros(m + 1, n + 1)

    # Fill first row and column with gap penalties
    for i in range(m + 1):
        score[i][0] = i * GAP_PENALTY
    for j in range(n + 1):
        score[0][j] = j * GAP_PENALTY

    # Fill the score matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score[i - 1][j - 1] + match_score(seq1[j - 1], seq2[i - 1])
            delete = score[i - 1][j] + GAP_PENALTY
            insert = score[i][j - 1] + GAP_PENALTY
            score[i][j] = max(match, delete, insert)

    return score


Define Backtrace Function

In [9]:
def nw_backtrace(seq1, seq2):
    """Backtraces the Needleman-Wunsch matrix to get the optimal alignment."""
    score = needleman_wunsch(seq1, seq2)

    align1 = []
    align2 = []

    i, j = len(seq2), len(seq1)

    while i > 0 and j > 0:
        score_current = score[i][j]
        score_diagonal = score[i - 1][j - 1]
        score_up = score[i][j - 1]
        score_left = score[i - 1][j]

        if score_current == score_diagonal + match_score(seq1[j - 1], seq2[i - 1]):
            align1.append(seq1[j - 1])
            align2.append(seq2[i - 1])
            i -= 1
            j -= 1
        elif score_current == score_up + GAP_PENALTY:
            align1.append(seq1[j - 1])
            align2.append('-')
            j -= 1
        elif score_current == score_left + GAP_PENALTY:
            align1.append('-')
            align2.append(seq2[i - 1])
            i -= 1

    while j > 0:
        align1.append(seq1[j - 1])
        align2.append('-')
        j -= 1
    while i > 0:
        align1.append('-')
        align2.append(seq2[i - 1])
        i -= 1

    return ''.join(reversed(align1)), ''.join(reversed(align2))


In [10]:
# Test sequences
seq1 = "GATGCV"
seq2 = "-GATCV"

# Compute scoring matrix
matrix = needleman_wunsch(seq1, seq2)

# Prepare row and column names
row_names = [''] + list(seq2)  # First row is empty, then sequence 2
col_names = [''] + list(seq1)  # First column is empty, then sequence 1

# Print scoring matrix with row and column names
print("Scoring Matrix with Row and Column Names:")
print_matrix(matrix, row_names, col_names)

# Compute and print alignments
alignment1, alignment2 = nw_backtrace(seq1, seq2)
print("\nFinal Alignment:")
print(alignment1)
print(alignment2)


Scoring Matrix with Row and Column Names:
            G     A     T     G     C     V
          0    -2    -4    -6    -8   -10   -12
    -    -2    -2    -4    -6    -8   -10   -12
    G    -4     8     6     4     4     2     0
    A    -6     6    18    16    14    12    10
    T    -8     4    16    28    26    24    22
    C   -10     2    14    26    24    36    34
    V   -12     0    12    24    22    34    46

Final Alignment:
-GATGCV
-GAT-CV
